In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# === Parametri ===
ticker_symbol = "^SPX"      # S&P 500
target_expiration = "2025-10-17"  # Scadenza opzioni
target_strike = 5000        # Strike desiderato (da regolare in base al livello attuale dell'SPX)

# === Scarica storico prezzi del sottostante ===
end_date = datetime.today()
start_date = end_date - timedelta(days=365)
underlying = yf.download(ticker_symbol, start=start_date, end=end_date, interval="1d")

# === Scarica catena opzioni ===
ticker = yf.Ticker(ticker_symbol)
opt_chain = ticker.option_chain(target_expiration)
calls = opt_chain.calls
puts = opt_chain.puts

# === Trova l'opzione Call e Put allo strike desiderato ===
try:
    call_symbol = calls.loc[calls['strike'] == target_strike, 'contractSymbol'].values[0]
    put_symbol = puts.loc[puts['strike'] == target_strike, 'contractSymbol'].values[0]
except IndexError:
    raise ValueError(f"Nessuna opzione trovata per strike {target_strike}. Strike disponibili: {calls['strike'].tolist()}")

# === Scarica storico prezzi opzioni ===
call_data = yf.download(call_symbol, start=start_date, end=end_date, interval="1d")
put_data = yf.download(put_symbol, start=start_date, end=end_date, interval="1d")

# === Grafico ===
fig, ax1 = plt.subplots(figsize=(12,6))

# Prezzi Call e Put
ax1.plot(call_data.index, call_data['Close'], label=f"Call {target_strike}", color='blue')
ax1.plot(put_data.index, put_data['Close'], label=f"Put {target_strike}", color='red')
ax1.set_xlabel("Data")
ax1.set_ylabel("Prezzo Opzioni ($)")
ax1.legend(loc="upper left")

# Prezzo sottostante su secondo asse
ax2 = ax1.twinx()
ax2.plot(underlying.index, underlying['Close'], label="Sottostante", color='green', linestyle="--")
ax2.set_ylabel("Prezzo Sottostante ($)")

plt.title(f"Andamento Call & Put Strike {target_strike} - Scadenza {target_expiration} - {ticker_symbol}")
plt.show()